In [1]:
%reload_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

pd.options.display.max_columns = 999
pd.options.display.max_rows = 1999
pd.options.display.max_colwidth = 200

from paths import *
from scpts import manipulation
from scpts import io
from scpts import caged

from basedosdados import Storage, Table, Dataset
import basedosdados as bd




### Download Dados
 - ftp://ftp.mtps.gov.br/pdet/microdados/CAGED/

In [7]:
raw_path = '../data/caged/raw/'
clean_path = '../data/caged/clean/'

anos = [i for i in range(2007,2020)]
meses= ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']

for ano in anos:
    for mes in meses:
        caged.download_caged_file(ano,mes,raw_path)

01/2007 ja existe
02/2007 ja existe
03/2007 ja existe
04/2007 ja existe
05/2007 ja existe
06/2007 ja existe
07/2007 ja existe
08/2007 ja existe
09/2007 ja existe
10/2007 ja existe
11/2007 ja existe
12/2007 ja existe
01/2008 ja existe
02/2008 ja existe
03/2008 ja existe
04/2008 ja existe
05/2008 ja existe
06/2008 ja existe
07/2008 ja existe
08/2008 ja existe
09/2008 ja existe
10/2008 ja existe
11/2008 ja existe
12/2008 ja existe
01/2009 ja existe
02/2009 ja existe
03/2009 ja existe
04/2009 ja existe
05/2009 ja existe
06/2009 ja existe
07/2009 ja existe
08/2009 ja existe
09/2009 ja existe
10/2009 ja existe
11/2009 ja existe
12/2009 ja existe
01/2010 ja existe
02/2010 ja existe
03/2010 ja existe
04/2010 ja existe
05/2010 ja existe
06/2010 ja existe
07/2010 ja existe
08/2010 ja existe
09/2010 ja existe
10/2010 ja existe
11/2010 ja existe
12/2010 ja existe
01/2011 ja existe
02/2011 ja existe
03/2011 ja existe
04/2011 ja existe
05/2011 ja existe
06/2011 ja existe
07/2011 ja existe
08/2011 ja

### Padronizacao

- o arquivo CAGED_062012 teve troca de nome quando foi descompactado
&nbsp;
- Particao | ano=XXXX/mex=XX/sigla_uf=XX

In [3]:

# ##--------------Download basedosdados---------------###
# q_municipios = """
# SELECT id_municipio,id_municipio_6,sigla_uf
# FROM `basedosdados.br_bd_diretorios_brasil.municipio`
# """
# municipios = bd.read_sql(q_municipios, billing_project_id='gabinete-sv')
# municipios.to_parquet(f'../data/caged/raw/municipios.parquet')


In [15]:
raw_path = '../data/caged/raw/'
clean_path = '../data/caged/clean/'

municipios = pd.read_parquet(f'{raw_path}municipios.parquet')

# anos = [ano for ano in os.listdir(raw_path) if len(ano)==4]
anos = [i for i in range(2007,2020)]
all_cols = pd.DataFrame()

for ano in anos:
    options = os.listdir(raw_path + f"{ano}")
    for option in options:
        meses = os.listdir(raw_path + f"{ano}/" + f"{option}/")
        meses = [int(m) for m in meses]
        meses.sort()
        meses = [str(m) for m in meses]
        for mes in meses:
            path_month = raw_path + f"{ano}/"+ f"{option}/" + f"{int(mes)}/"
            ## limpa residuo de arquivos inacabados
            filename_7z = caged.get_file_names_and_clean_residues(path_month)
            
            ## verifica se o arquivo ja foi tratado
            if os.path.exists(f'{clean_path}/ano={ano}/mes={mes}') and \
               len(os.listdir(f'{clean_path}/ano={ano}/mes={mes}'))==27:
                print(f'{ano}-{mes} --> ja tratado\n')
                pass
            else:
                ## extrai o arquivo zipado, cria um novo arquivo .csv e deleta o arquivo extraido (.txt)
                caged.extract_file(path_month,filename_7z,save_rows=None)
                print(f'{ano}-{mes} --> extraido')
                
                ## le o arquivo 
                filename = [file for file in os.listdir(path_month) if '.csv' in file][0][:-4]
                df = pd.read_csv(f'{path_month}{filename}.csv')

                ### padronizacao dos dados
                df = caged.padroniza_caged(df,municipios)
                print(f'{ano}-{mes} --> padronizado')
                print(f'{ano}-{mes} -->',df['ano'].unique())
                print(f'{ano}-{mes} -->',df['mes'].unique())
                
                ### salva arquivo na estrutura de particoes
                for uf in df.sigla_uf.unique():
                    ## filtra apenas o estado de interesse
                    df_uf = df[df['sigla_uf']==uf]
                    ## exclui colunas particionadas
                    df_uf = df_uf.drop(['ano','mes','sigla_uf'],1)
                    ## cria estrutura de pastas
                    path_uf = caged.make_folder_tree(clean_path,ano,mes,uf)
                    df_uf.to_csv(f'{path_uf}caged.csv', index=False)



                ##remove arquivo csv do raw files
                os.remove(f'{path_month}{filename}.csv')
                print(f'{ano}-{mes} --> finalizado\n')

                #checa nome de todas as colunas
                dd = pd.DataFrame(df.columns.tolist(),columns=['cols'])
                dd = dd.transpose().reset_index(drop=True)
                cols = dd.columns.tolist()
                dd['ano']=ano
                dd['mes']=mes
                dd['option']=option

                dd = dd[['ano','mes','option'] + cols]

                all_cols = pd.concat([all_cols,dd])


2007-1 --> ja tratado

2007-2 --> ja tratado

2007-3 --> ja tratado

2007-4 --> ja tratado

2007-5 --> ja tratado

2007-6 --> ja tratado

2007-7 --> ja tratado

2007-8 --> ja tratado

2007-9 --> ja tratado

2007-10 --> ja tratado

2007-11 --> ja tratado

2007-12 --> ja tratado

2008-1 --> ja tratado

2008-2 --> ja tratado

2008-3 --> ja tratado

2008-4 --> ja tratado

2008-5 --> ja tratado

2008-6 --> ja tratado

2008-7 --> ja tratado

2008-8 --> ja tratado

2008-9 --> ja tratado

2008-10 --> ja tratado

2008-11 --> ja tratado

2008-12 --> ja tratado

2009-1 --> ja tratado

2009-2 --> ja tratado

2009-3 --> ja tratado

2009-4 --> ja tratado

2009-5 --> ja tratado

2009-6 --> ja tratado

2009-7 --> ja tratado

2009-8 --> ja tratado

2009-9 --> ja tratado

2009-10 --> ja tratado

2009-11 --> ja tratado

2009-12 --> ja tratado

2010-1 --> ja tratado

2010-2 --> ja tratado

2010-3 --> ja tratado

2010-4 --> ja tratado

2010-5 --> ja tratado

2010-6 --> ja tratado

2010-7 --> ja tratado

20

In [ ]:
data = pd.read_csv('file1.csv', )

In [2]:
# microdados_novo_caged_vinculos e microdados_novo_caged_estabelecimentos
tb = Table('microdados_caged','br_me_caged')

In [3]:
# tb.create(
#     path='../data/caged/clean/2007/',
#     partitioned=True
# )

In [4]:
for ano in range(2008,2020):
    tb.append(
        filepath=f'../data/caged/clean/{ano}/',
        partitions='ano=value/mes=value/sigla_uf=value',
        if_exists='replace'
    )
    print(f'{ano} done!\n')

Uploading files:   0%|          | 0/324 [00:00<?, ?it/s]

2008 done!



Uploading files:   0%|          | 0/324 [00:00<?, ?it/s]

2009 done!



Uploading files:   0%|          | 0/324 [00:00<?, ?it/s]

...

2017 done!



Uploading files:   0%|          | 0/324 [00:00<?, ?it/s]

2018 done!



Uploading files: 100%|██████████| 324/324 [29:49<00:00,  5.52s/it] 


2019 done!

